In [1]:
# Logistic Regression

Импортируем наши данные ис незначительными изменениям: зависимую переменную Price преобразуем в виде показателя, состоящего из 2х значений: 0 и 1. Деление происходило по принципу поиска целого среднего =14661.  Все значения, которые меньше этого значения получили кодировку "0", все значения, которые бальше, полчуили кожировку "1". Проверим, как отобразились наши данные.  

In [2]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# Importing the dataset
df = pd.read_csv('vehicleprice_prep2.csv')

In [4]:
df

,brand,model,year,body,car_mileage,fuel,power,transmission,price
0,ford,transit,2006,other,220,diesel,2.0,manual,0
1,renault,master,2017,other,211,diesel,2.3,manual,1
2,opel,vivaro,2015,other,157,diesel,1.6,manual,1
3,ford,transit,2000,other,300,diesel,2.4,manual,0
4,volkswagen,t4,2002,other,270,diesel,2.5,manual,0
...,...,...,...,...,...,...,...,...,...
1935,renault,laguna,2005,liftback,311,diesel,1.9,manual,0
1936,skoda,octavia,2009,liftback,201,diesel,2.0,manual,0
1937,audi,a5,2012,liftback,199,petrol,1.8,automatic,1
1938,skoda,octavia,2008,liftback,226,gas,1.6,manual,0


Убедимся в отсутствии пропущенных значений. Как видим, они отсутствуют.

In [5]:
# Cheking Missing data
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data

,Total,Percent
price,0,0.0
transmission,0,0.0
power,0,0.0
fuel,0,0.0
car_mileage,0,0.0
body,0,0.0
year,0,0.0
model,0,0.0
brand,0,0.0


В нашем датасете присутствуют качественные переменные. Их необходимо закодировать с помощью функции.
Запишем список нечисловых переменных и проведем кодирование качественных переменных. Результат запишем обратно в базу.

In [7]:
# Function Encoding
def encoding_char(x):
    char_var = list(set(x.columns) - set(x._get_numeric_data().columns))
    for col_names in char_var:
        f = pd.factorize(x[col_names])
        x[col_names] = pd.factorize(x[col_names])[0]
    return(x)

In [8]:
# Encoding categorical data
df = encoding_char(df)

In [9]:
df

,brand,model,year,body,car_mileage,fuel,power,transmission,price
0,0,0,2006,0,220,0,2.0,0,0
1,1,1,2017,0,211,0,2.3,0,1
2,2,2,2015,0,157,0,1.6,0,1
3,0,0,2000,0,300,0,2.4,0,0
4,3,3,2002,0,270,0,2.5,0,0
...,...,...,...,...,...,...,...,...,...
1935,1,98,2005,10,311,0,1.9,0,0
1936,10,13,2009,10,201,0,2.0,0,0
1937,12,157,2012,10,199,1,1.8,1,1
1938,10,13,2008,10,226,2,1.6,0,0


Разделим наш датасет на обучающую и тестовую выборку по пропорции 20% на 80%

In [11]:
# Splitting the dataset into the Training set and Test set
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

Проведем шкалирование данных. Из библиотеки импортируем класс StandardScaler и рассчитаем и запишем в переменную необходимые 
параметры шкалирования, посчитанные на основе обучающей выборки. Затем с использованием жтих параметров шкалируем обучающую и 
тестовую выборкую. Эндогенная переменная в шкалировании не нуждается. Она должна быть дискретного типа 0;1. Что у нас и есть.

In [12]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler().fit(X_train)
X_train = sc_X.transform(X_train)
X_test = sc_X.transform(X_test)

Строим базовую модель. Для жтого из библиотеки statsmodels с помощью ф-и Logit посмотрим отчет по модели. Видим, что все 
переменные значимыми, кроме Х1 и Х4; т.к. их значения Р не превышают 3%. Но, полагаясь на наш прошлый опыт того, что удаление из модели незнчимых переменных не даст улучшения, имеем право оставить все переменные.

In [13]:
# Baseline model
import statsmodels.api as sm
lr = sm.Logit(y_train, X_train).fit()
print(lr.summary2())

Optimization terminated successfully.
         Current function value: 0.360051
         Iterations 8
                          Results: Logit
Model:              Logit            Pseudo R-squared: 0.428      
Dependent Variable: y                AIC:              1133.5983  
Date:               2020-12-20 17:21 BIC:              1176.3767  
No. Observations:   1552             Log-Likelihood:   -558.80    
Df Model:           7                LL-Null:          -976.16    
Df Residuals:       1544             LLR p-value:      5.9552e-176
Converged:          1.0000           Scale:            1.0000     
No. Iterations:     8.0000                                        
---------------------------------------------------------------------
        Coef.     Std.Err.       z       P>|z|      [0.025     0.975]
---------------------------------------------------------------------
x1      0.0269      0.0832     0.3237    0.7462    -0.1361     0.1900
x2      0.3105      0.0945     3.2843    

Проведем автоматический отбор переменных для того, чтгобы убдедиться в правильности нашего выбора. Как видим, наше предположение о вохможности оставить все переменные оказлось правильным.

In [14]:
# Automatic Feature Selection
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
estimator=LogisticRegression(random_state=123)
selector=RFECV(estimator, step=1)
selector=selector.fit(X_train, y_train)
print(selector.n_features_)
print(selector.support_)
print(selector.ranking_)

8
[ True  True  True  True  True  True  True  True]
[1 1 1 1 1 1 1 1]


Выведем индексы отобранных переменных (необязательное действие)

In [15]:
# Selected Features
selected_columns=[]
for i in range(len(X_train[0])):
    if selector.support_[i]==1:
        selected_columns.append(i)
print(selected_columns)
print('№\tIndex\tFeature')
i=0
for column in selected_columns:
    print(str(i)+'\t'+str(column)+'\t'+str(df.columns[column]))
    i=i+1

[0, 1, 2, 3, 4, 5, 6, 7]
№	Index	Feature
0	0	brand
1	1	model
2	2	year
3	3	body
4	4	car_mileage
5	5	fuel
6	6	power
7	7	transmission


Преобразуем обучающую и тестовую выборки, оставив в ней только значимые переменные. Как и было сказано изначально и показано автоматическим отбором, мы имеем право оставить все переменные, поэтому используем эту возможность.

In [14]:
# p-value < 1% Features
X_train = X_train[:,[0,1,2,3,4,5,6,7]]
X_test = X_test[:,[0,1,2,3,4,5,6,7]]

Обучим модель логистическй регрессии на 8 переменных.

In [16]:
# Fitting Logistic Regression to the Training set (2 variables)
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state = 13).fit(X_train, y_train)

Проведем прогноз на тестовой выборке и оценим качетво модели. Уровень качества высокий, 87,37% объектов распознаны верно.

In [17]:
# Predicting the Test set results
y_pred = lr.predict(X_test)
lr.score(X_test,y_test)

0.8737113402061856

Выведем таблицу сопряженности, которая показывает, что 20 позитивных объектов распознаны как негативные, а 29
негативных определены как позитивные.

In [19]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[241  29]
 [ 20  98]]


Произведем расчет доли ошибки, чувствителности и специфичности модели.

In [20]:
Error_rate = (20+29)/(388)
Error_rate

0.12628865979381443

In [21]:
Se = 98/(98+20)
Se

0.8305084745762712

In [22]:
Sp = 241/(241+29)
Sp

0.8925925925925926

Итак, можно увидеть, что в модели логистической регрессии чувствительность составляет 83%, специфичность 89%.Это говорит, что модель приблизительно одинаково пределяет как определяет истинно негативные, так и истинно позитивные случаи.
Доля ошибки низкая = 12,6%, точность высокая = 87,37%.
Это наш базовый уровень для построения Нейронной Сети.

К сожалению,визуализировать данные не представляется возможным из-за количества взятых переменных. Экспортируем полученные данные.

In [23]:
import pickle
obj = {'X_train': X_train, 'X_test': X_test,'y_train': y_train,'y_test': y_test}
output = open('vehicleprice_data.pkl', 'wb')
pickle.dump(obj, output, 2)
output.close()